In [1]:
# this is required ONLY in a Jupyter Notebook
# otherwise you'll get a "RuntimeError: Cannot run the event loop while another loop is running"
# omit this import and apply in the real code
import nest_asyncio

nest_asyncio.apply()  # can be omitted outside jupyter

In [2]:
from ahbicht.expressions.ahb_expression_parser import parse_ahb_expression_to_single_requirement_indicator_expressions

tree = parse_ahb_expression_to_single_requirement_indicator_expressions(
    "Muss [2] U ([3] O [4])[901] U [555]"
)  # the expression as you get it from the AHB
print(tree)  # The tree is a data structure that represents the expression

# you may also serialize it as JSON:
from ahbicht.json_serialization.tree_schema import TreeSchema

TreeSchema().dump(tree)

Tree('ahb_expression', [Tree('single_requirement_indicator_expression', [Token('MODAL_MARK', 'Muss'), Token('CONDITION_EXPRESSION', ' [2] U ([3] O [4])[901] U [555]')])])


{'type': 'ahb_expression',
 'children': [{'tree': {'type': 'single_requirement_indicator_expression',
    'children': [{'tree': None, 'string': 'Muss'},
     {'tree': None, 'string': ' [2] U ([3] O [4])[901] U [555]'}]},
   'string': None}]}

In [3]:
# As written in the README, ahbicht will _not_ do any content evaluation for you.
# You'll need to write your own Evaluator classes for:
# 1. Requirement Constraints (RC) for conditions which are fulfilled/unfulfilled/unknown
# 2. Format Constraints (FC) which describe that data must obey a specified format, e.g. number of post decimal places, MaLo-ID etc.
# 3. Hints: "Hinweise", plain text which is passed through

In [4]:
from ahbicht.content_evaluation.rc_evaluators import RcEvaluator

from ahbicht.edifact import EdifactFormat, EdifactFormatVersion
from ahbicht.content_evaluation.evaluationdatatypes import EvaluationContext, EvaluatableData


class MyRequirementConstraintEvaluator(RcEvaluator):
    """
    A Requirement Constraint (RC) Evaluator answers questions of the kind: "Is the condition <condition_number> required?"
    """

    def _get_default_context(self) -> EvaluationContext:
        # implement a default context here
        return EvaluationContext(scope=None)

    # in the AHBs the conditions are unique per EDIFACT format / Message Implementation Guide
    edifact_format = EdifactFormat.UTILMD
    edifact_format_version = EdifactFormatVersion.FV2104  # valid since 2021-04-01

    def __init__(self, evaluatable_data):
        super().__init__(evaluatable_data=evaluatable_data)

    async def evaluate_2(self, context: EvaluationContext):
        # insert your own logic here
        return self.evaluatable_data.edifact_seed["2"]

    async def evaluate_3(self, context: EvaluationContext):
        # insert your own logic here
        return self.evaluatable_data.edifact_seed["3"]

    async def evaluate_4(self, context: EvaluationContext):
        # insert your own logic here
        return self.evaluatable_data.edifact_seed["4"]

In [5]:
from ahbicht.expressions.condition_nodes import ConditionFulfilledValue, EvaluatedFormatConstraint
from ahbicht.content_evaluation.fc_evaluators import FcEvaluator


class MyFormatConstraintEvaluator(FcEvaluator):
    """
    Format Constraint (FC) Evaluator answers questions of the kind: "Does the data provided obey format specified in condition <condition_number>?"
    """

    edifact_format = EdifactFormat.UTILMD
    edifact_format_version = EdifactFormatVersion.FV2104  # valid since 2021-04-01

    async def evaluate_901(self, entered_input):
        """
        This method checks if the entered_input fulfills the constraints of condition 901.
        This could be f.e. if entered_input is a valid OBIS, a valid Marktlokations-ID etc.
        For this MWE we check if the input is all lower case.
        """
        # insert your own logic here
        if entered_input["data"].lower() == entered_input["data"]:
            return EvaluatedFormatConstraint(True, None)
        return EvaluatedFormatConstraint(
            False, f"The input '{entered_input['data']}' does not obey format constraint 901."
        )

In [6]:
from ahbicht.expressions.hints_provider import HintsProvider


class MyHintsProvider(HintsProvider):
    """
    A Hints Provider provides plain text for given "condition" numbers.
    """

    edifact_format = EdifactFormat.UTILMD
    edifact_format_version = EdifactFormatVersion.FV2104

    async def get_hint_text(self, condition_key: str):
        # implement your own logic here
        if condition_key == "555":
            return "Hinweis 555 applies."
        return None

In [7]:
# we'll just provide some hard coded data here for demonstration purposes
hardcoded_content_evaluations = {
    "2": ConditionFulfilledValue.FULFILLED,
    "3": ConditionFulfilledValue.UNFULFILLED,
    "4": ConditionFulfilledValue.FULFILLED,
    "data": "some lower case stuff which will be checked in 901",
}

my_evaluatable_data = EvaluatableData(
    edifact_seed=hardcoded_content_evaluations
)  # this is the data that, in real life, contain the content of the edifact message

In [8]:
import inject
from ahbicht.expressions.ahb_expression_evaluation import evaluate_ahb_expression_tree

inject.clear_and_configure(
    lambda binder: binder.bind(RcEvaluator, MyRequirementConstraintEvaluator(evaluatable_data=my_evaluatable_data))
    .bind(FcEvaluator, MyFormatConstraintEvaluator())
    .bind(HintsProvider, MyHintsProvider())
)

# But later on we can provide AHBicht with the content evaluation results ...
# Providing the content evaluation results to find out, if a line in the AHB is actually required,
# is called expression evaluation
expression_evalution_result = evaluate_ahb_expression_tree(tree, entered_input=hardcoded_content_evaluations)

In [9]:
print(f"It's actually a '{expression_evalution_result.requirement_indicator}'field.")
if expression_evalution_result.format_constraint_evaluation_result.format_constraints_fulfilled:
    print(f"The format constraint is fulfilled => Data are ok.")
else:
    print(
        f'The format constraint is _not_ fulfilled: "{expression_evalution_result.format_constraint_evaluation_result.error_message}"'
    )
print(f'Please note the following hint: "{expression_evalution_result.requirement_constraint_evaluation_result.hints}"')

It's actually a 'Muss'field.
The format constraint is fulfilled => Data are ok.
Please note the following hint: "Hinweis 555 applies."
